In [1]:
import tensorflow as tf
import numpy as np

In [2]:
text = open('./data/sentiment labelled sentences/amazon_cells_labelled.txt', 'r').read()

In [3]:
text[:20]

'So there is no way f'

In [4]:
vocab = set(text) # Vocabulary, the set of characters in our text corpus

In [5]:
# Create map between characters and integer indices

char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(list(vocab))

In [6]:
# char2idx['p']

In [7]:
# idx2char[72]

### Create training and target examples

In [8]:
# Trainig parameters
seq_length = 50
examples_per_epoch = len(text)//seq_length

# We need to encode the full text list into integers
text_as_int = np.array([char2idx[c] for c in text])

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [9]:
for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

S
o
 
t
h


In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [11]:
# We use the batch method of char_dataset to generate training batches
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [12]:
dataset = sequences.map(split_input_target)

In [13]:
for input_example, target_example in dataset.take(1):
    print("Input: ", ''.join(idx2char[input_example.numpy()]))
    print("Output: ", ''.join(idx2char[target_example.numpy()]))

Input:  So there is no way for me to plug it in here in th
Output:  o there is no way for me to plug it in here in the


In [14]:
final_dataset = dataset.shuffle(5000).batch(64, drop_remainder=True)

## Build the model

In [15]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024 # Recurrent neural network units

**IMPORTANT**: We need two different modes here, inference and training, and to build the model in two different ways.

- For training, we want our model to take batches of several (input_example, target_example) pairs.
- For inference, we want to start with a single seed string, and get it writing from that point on.

In [16]:
def build_model(batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_size=batch_size), # Specify batch dimensions and embedding (reshape)
        tf.keras.layers.GRU(rnn_units,
                            return_sequences = True,
                            stateful = True), 
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [17]:
#for input_example, target_example in final_dataset.take(1):
#    example_batch_preds = model(input_example)
#    print(example_batch_preds)

In [18]:
def loss(labels, probas):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, probas, from_logits=True)


In [19]:
model = build_model(batch_size=64)

In [20]:
model.compile(optimizer='adam', loss=loss)

In [21]:
model.fit(final_dataset, epochs=100)

Epoch 1/100
17/17 [==============================] - 22s 1s/step - loss: 4.2791
Epoch 2/100
17/17 [==============================] - 24s 1s/step - loss: 3.4196
Epoch 3/100
17/17 [==============================] - 25s 1s/step - loss: 2.9455
Epoch 4/100
17/17 [==============================] - 24s 1s/step - loss: 2.6201
Epoch 5/100
17/17 [==============================] - 25s 1s/step - loss: 2.4302
Epoch 6/100
17/17 [==============================] - 25s 1s/step - loss: 2.3359
Epoch 7/100
17/17 [==============================] - 29s 2s/step - loss: 2.2692
Epoch 8/100
17/17 [==============================] - 31s 2s/step - loss: 2.2150
Epoch 9/100
17/17 [==============================] - 29s 2s/step - loss: 2.1650
Epoch 10/100
17/17 [==============================] - 31s 2s/step - loss: 2.1187
Epoch 11/100
17/17 [==============================] - 31s 2s/step - loss: 2.0636
Epoch 12/100
17/17 [==============================] - 29s 2s/step - loss: 2.0078
Epoch 13/100
17/17 [=================

In [22]:
def generate_text(model, start_string):
    # Check what our model is doing. start_string will be a text string
    
    num_generate = 1000 # number of characters that the model will generate
    
    # First, we vectorize our start string
    input_vec = [char2idx[s] for s in start_string]
    input_vec = tf.expand_dims(input_vec, 0) # make dimension consistent with tf
    
    text_generated = [] # here goes the generated text
    
    temperature = 1.0 # High values will yield more surprising text :)
    
    for i in range(num_generate):
        predictions = model(input_vec)
        predictions = tf.squeeze(predictions, 0) # removes the batch dimension
        
        # Rescale by temp, and make a draw for the next-char id's
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        # Finally, pass this id to idx2char
        text_generated.append(idx2char[predicted_id])
        
        # Add the new character and pass to the next step in the loop
        input_vec = tf.expand_dims([predicted_id], 0)
    
    return start_string + ''.join(text_generated)

In [23]:
model.save_weights('rnn.h5')

In [24]:
model = build_model(batch_size=1)

In [25]:
model.load_weights('rnn.h5')

In [26]:
model.build(tf.TensorShape([1,None]))

In [27]:
generate_text(model, start_string='I bought this product ')

'I bought this product is great.\t1\nI have to jightly and is sturdy on the market yet.\t1\nSWEETEST PHONE!!!\t1\n:-) in unyone!.\t1\nI wouldn\'t make the same mistake I did.\t0\nOh, the better run out favadit on and the sound is much better than otherso use because case.\t0\nIt\'s AGGRAVATI got weak snap!\t0\nDoes not charge the Cingular (ATT) 853 my calls and carries the highest quality of anti-glare se with no problem with this item and would order it again if needed.\t1\nYou get what you pay for.\t0\nThe handsfree part what you pay for product.\t0\nHow can that be?The audio quality and weil better funnt, the menus and dirction feels good.\t1\nI received my headset in good time and with no stars because it toucher contacting the company for few dollar product is very Highfand noticed than inside.\t1\nA lot of websites have been satisife.\t0\nIt\'s not what it says it is.\t0\nBut now that it is "outyour laptou plan to use this in a car forgets super charged up for use as a small hybr